In [33]:
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [34]:
def processing_fasta_file(path, PTM):
    """
    Parameters
    ----------
    path: string
        path to .fasta file with sequences
    PTM: list of strings
        the phosphorylation site such as ['S', 'T']
        
    Returns
    -------
    id_phos_sites: numpy array
        array of UniProtIDs and phosphorylation indices
    phos_dict : dict
        dictionary with Uniprot IDs as keys and phosphorylation sequences 
    
    """
    fp = open(path, 'r+')
    lines = []
    with fp as f:
        lines = f.readlines()

    ID = []
    uniprot_ID = []
    c = []
    indx = []
    p_ind = []
    p = []

    for i in range(0, len(lines), 2):
        ID.append(lines[i])

    for i in range(len(ID)):
        uniprot_ID.append(ID[i].split("|"))

    for lis in range(len(uniprot_ID)):
        for i in range(1, len(uniprot_ID[lis]), 2):
            c.append(uniprot_ID[lis][i])
    phosp = np.array(c)

    ID = []
    def split(word):
        return list(word)

    for i in range(1, len(lines), 2):
        ID.append([lines[i]])

    for lis in range(len(ID)):
        for char in range(len(ID[lis])):
            indx.append(split(ID[lis][char]))

    for lis in range(len(indx)):
        p = []
        for i in range(len(indx[lis])):
            for k in range(len(PTM)):
                if indx[lis][i] == PTM[k]:
                    p.append(i)
        p_ind.append(tuple(p))
    
    p_ind = np.array(p_ind, dtype=tuple)
    print(f"p_ind: {p_ind}")
    print(f"phosp: {phosp}")
    if len(p_ind) == 1:
        id_phos_sites = np.zeros(shape=(1,2), dtype = object)
        id_phos_sites[:,0] = phosp
        id_phos_sites[:,1] = tuple(p_ind)
    else:
        id_phos_sites = np.vstack((phosp, p_ind)).T
    phos_dict = dict(np.c_[phosp, ID])
    print(f"id_phos_sites: {id_phos_sites}")
    print(f"phos_dict: {phos_dict}")
    return id_phos_sites, phos_dict

In [35]:
def kmers(k, phos_dict, PTM):
    """

    Parameters
    ----------
    k : int
        an odd number that specifies that length of the kmer with the
        phosphorylation site in the middle
    phos_dict : dict
        dictionary with Uniprot IDs as keys and phosphorylation sequences 
    PTM: list of strings
        the phosphorylation site such as ['S', 'T']

    Returns
    -------
    kmer : 2D array
        2D array with columns UniProt ID, phosphorylation index, k-mer

    """
    kmer = []
    ID = list(phos_dict.keys())
    for q in tqdm(range(len(ID)), desc="Kmers", position=0, leave=True):
        for d in range(len(phos_dict[ID[q]])):
            for t in range(len(PTM)):
                if phos_dict[ID[q]][d] == PTM[t]:
                    around_ind = int((k-1)/2)
                    if d-around_ind < 0 and d+around_ind+1 < len(phos_dict[ID[q]]):
                        c = d-0
                        g = phos_dict[ID[q]][d-c:d+around_ind+1]
                        kmer.append([ID[q], d, "-"*(around_ind-d) + g])
                    elif d-around_ind >= 0 and d+around_ind+1 >= len(phos_dict[ID[q]]):
                        c = len(phos_dict[ID[q]]) - d
                        g = phos_dict[ID[q]][d-around_ind:d+c]
                        kmer.append([ID[q], d, g + "-"*(d+around_ind+1-len(phos_dict[ID[q]]))])
                    elif d-around_ind < 0 and d+around_ind+1 >= len(phos_dict[ID[q]]):
                        c = d-0
                        b = len(phos_dict[ID[q]]) - d
                        g = phos_dict[ID[q]][d-c:d+b]
                        kmer.append([ID[q], d, "-"*(around_ind-d) + g + "-"*(d+around_ind+1-len(phos_dict[ID[q]]))])
                    else:
                        kmer.append([ID[q], d, phos_dict[ID[q]][d-around_ind:d+around_ind+1]])
    
    kmer = np.array(kmer)
    return kmer

In [36]:
def sequence_predict(sequence_path, modification, cutoff):
    """
    Parameters
    ----------
    sequence_path: string
        path to .fasta file with sequences
    modification: string
        phos, glycosylation, hydroxyproline, CDK, CK2, MAPK, PKA, or PKC 
    cutoff: float
        number between 0 and 1 to add PTM labels to k-mers
        
    Returns
    -------
    probability: numpy array
        array with PTM indices and probability
    
    """
    if modification == "glycosylation":
        PTM = ['N']
    elif modification == "hydroxyproline":
        PTM = ['P']
    else:
        PTM = ['S', 'T']
    
    id_phos_sites, phos_dict = processing_fasta_file(sequence_path, PTM)
    kmer = kmers(35, phos_dict, PTM)
    # One-hot encoding
    alphabet = "ARNDCEQGHILKMFPSTWYV-U"

    def convert_to_onehot(data):
        #Creates a dict, that maps to every char of alphabet an unique int based on position
        global char_to_int
        char_to_int = dict((c,i) for i,c in enumerate(alphabet))
        encoded_data = []
        #Replaces every char in data with the mapped int
        encoded_data.extend([char_to_int[char] for char in data])
        return encoded_data


    def tensor_encoding(k, x_data, depth):
        indices = []
        t2 = []
        for i in range(len(x_data)):
            indices.append(convert_to_onehot(x_data[i,2]))
        for i in range(len(indices)):
            t1 = tf.one_hot(indices[i], depth)
            t2.append(t1)
        return t2

    tensor = tensor_encoding(35, kmer, 22)
    np.save('dataset', tensor)
    dataset = np.load('dataset.npy', allow_pickle=True).astype('float32')
    dataset = tf.convert_to_tensor(dataset)
    dataset = dataset.reshape(dataset.shape[0], 35, 22, 1)

    # Recreate the exact same model, including its weights and the optimizer
    new_model = tf.keras.models.load_model(f'{modification}_no_labels.h5')

    # Show the model architecture
    new_model.summary()

    predictions = new_model.predict(dataset)
    
    if PTM == ['S', 'T']:
        for i in range(len(predictions)):
            if predictions[i] > cutoff:
                if phos_dict[kmer[i,0]][int(kmer[i,1])] == "S":
                    phos_dict[kmer[i,0]] = phos_dict[kmer[i,0]][0:int(kmer[i,1])]+"X"+phos_dict[kmer[i,0]][int(kmer[i,1])+1:]        
                elif phos_dict[kmer[i,0]][int(kmer[i,1])] == "T":
                    phos_dict[kmer[i,0]] = phos_dict[kmer[i,0]][0:int(kmer[i,1])]+"Z"+phos_dict[kmer[i,0]][int(kmer[i,1])+1:]        
        
        kmer = kmers(35, phos_dict, ['S', 'T', 'X', 'Z'])

        # One-hot encoding
        alphabet = "ARNDCEQGHILKMFPSTWYVXZ-U"

        tensor = tensor_encoding(35, kmer, 24)
        np.save('dataset', tensor)
        dataset = np.load('dataset.npy', allow_pickle=True).astype('float32')
        dataset = tf.convert_to_tensor(dataset)
        dataset = dataset.reshape(dataset.shape[0], 35, 24, 1)
    else:
        for i in range(len(predictions)):
            if predictions[i] > cutoff:
                if phos_dict[kmer[i,0]][int(kmer[i,1])] == PTM[0]:
                    phos_dict[kmer[i,0]] = phos_dict[kmer[i,0]][0:int(kmer[i,1])]+"X"+phos_dict[kmer[i,0]][int(kmer[i,1])+1:]        
                
        kmer = kmers(35, phos_dict, [PTM[0], 'X'])

        # One-hot encoding
        alphabet = "ARNDCEQGHILKMFPSTWYVX-U"

        tensor = tensor_encoding(35, kmer, 23)
        np.save('dataset', tensor)
        dataset = np.load('dataset.npy', allow_pickle=True).astype('float32')
        dataset = tf.convert_to_tensor(dataset)
        dataset = dataset.reshape(dataset.shape[0], 35, 23, 1)       
    

    # Recreate the exact same model, including its weights and the optimizer
    new_model = tf.keras.models.load_model(f'{modification}_with_labels.h5')

    # Show the model architecture
    new_model.summary()

    predictions = new_model.predict(dataset)
    probability = np.c_[kmer[:,1], predictions]
    return probability

In [37]:
print(processing_fasta_file('tests/resources/two_sequences.fasta', ["S","T"]))

p_ind: [(5, 25, 30, 42, 44, 45, 47, 48, 49, 50, 51) (6, 7, 20, 30, 34, 35) ()
 (8, 12, 13, 27, 30, 33, 40, 42, 52, 53, 55)]
phosp: []


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 1 has size 4